In [68]:
import pandas as pd
from datetime import timedelta

df = pd.read_csv("logs.csv")
df = df[["email","created_at",'login']]
df.head()


email                created_at login
0  sethi.sankalp@karam.in  2020-08-18T12:05:49.304Z  True
1         mayank@karam.in  2020-08-17T05:02:51.465Z  True
2      dhanirams@karam.in  2020-08-13T09:15:56.796Z  True
3      rohitrana@karam.in  2020-08-12T03:57:54.273Z  True
4  sethi.sankalp@karam.in  2020-08-11T05:46:17.651Z   NaN

In [69]:
dates = []
for i in list(df['created_at']):
    dates.append(i.split("T")[0])

time = []
for i in list(df['created_at']):
    t = i.split("T")[1].split(".")[0]
    time.append(t)
    
df['Time'] = time
df['Date'] = dates
df['Date'] = pd.to_datetime(df['Date'])
df[['Hour','Minutes','Seconds']] = df['Time'].astype(str).str.split(':', expand=True).astype(int)
df.drop(columns=['created_at'],inplace=True)
df["Week"] = pd.DatetimeIndex(df['Date']).week
df['Year'] = pd.DatetimeIndex(df['Date']).year
df = df.sort_values('Date',ascending=True)
df.head()

email login      Time       Date  Hour  Minutes  Seconds  \
1532  dashboard@demo.com  True  06:45:42 2019-01-21     6       45       42   
1527  dashboard@demo.com   NaN  11:23:11 2019-01-21    11       23       11   
1528  dashboard@demo.com  True  10:16:30 2019-01-21    10       16       30   
1531  dashboard@demo.com  True  08:22:12 2019-01-21     8       22       12   
1530  dashboard@demo.com   NaN  08:26:43 2019-01-21     8       26       43   

      Week  Year  
1532     4  2019  
1527     4  2019  
1528     4  2019  
1531     4  2019  
1530     4  2019

In [70]:
# from dateutil.parser import parse
# startdates = []
# enddates = []
# def get_start_end_dates(year,week):
#     first_day_year = str(year) + '-' +  '01' + '-' + '01'
#     d = parse(first_day_year)
#     if(d.weekday()<= 3):
#         d = d - timedelta(d.weekday())             
#     else:
#         d = d + timedelta(7-d.weekday())
#     dlt = timedelta(days = (int(week)-1)*7)
#     startdates.append((d + dlt).strftime('%Y-%m-%d'))
#     enddates.append((d + dlt + timedelta(days=6)).strftime('%Y-%m-%d'))

In [71]:
# for (a,b) in zip(df['Year'],df['Week']):
#     get_start_end_dates(a,b)
    
# df['StartDate'] = startdates
# df['EndDate'] = enddates

import numpy as np

df['login'] = df['login'].replace(np.NaN,"Logout")
df = df[df['login']==True]
df.head()

email login      Time       Date  Hour  Minutes  Seconds  \
1532  dashboard@demo.com  True  06:45:42 2019-01-21     6       45       42   
1528  dashboard@demo.com  True  10:16:30 2019-01-21    10       16       30   
1531  dashboard@demo.com  True  08:22:12 2019-01-21     8       22       12   
1529  dashboard@demo.com  True  08:26:47 2019-01-21     8       26       47   
1525  dashboard@demo.com  True  12:25:19 2019-01-22    12       25       19   

      Week  Year  
1532     4  2019  
1528     4  2019  
1531     4  2019  
1529     4  2019  
1525     4  2019

In [59]:
grouped = df.groupby(['Date','email'])['Hour','Minutes','Seconds'].sum().reset_index()
grouped.head()

c:\users\karti\miniconda3\envs\kartik\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Date               email  Hour  Minutes  Seconds
0 2019-01-21  dashboard@demo.com    32      109      131
1 2019-01-22  dashboard@demo.com    18       83       52
2 2019-01-23  dashboard@demo.com     9       47       27
3 2019-01-24  dashboard@demo.com    10       40        5
4 2019-01-25  dashboard@demo.com    22       74       85

In [60]:
grouped['Hour'] = grouped['Hour'].apply(lambda x:x*3600)
grouped['Minutes'] = grouped['Minutes'].apply(lambda x: x*60)
grouped['TotalSeconds'] = [a+b+c for a,b,c in zip(grouped['Hour'],grouped['Minutes'],grouped['Seconds'])]
grouped.drop(columns=['Hour','Minutes','Seconds'],axis=1,inplace=True)

In [61]:
grouped.head()

Date               email  TotalSeconds
0 2019-01-21  dashboard@demo.com        121871
1 2019-01-22  dashboard@demo.com         69832
2 2019-01-23  dashboard@demo.com         35247
3 2019-01-24  dashboard@demo.com         38405
4 2019-01-25  dashboard@demo.com         83725

In [62]:
def convert(data):
    time = []
    for i in data['TotalSeconds']:     
        mini, sec = divmod(i, 60)
        hour, mini = divmod(mini, 60)
        t = "%d:%02d:%02d" % (hour, mini, sec)
        time.append(t)
    return time 
grouped['TimeSpent'] = convert(grouped)
grouped.head()

Date               email  TotalSeconds TimeSpent
0 2019-01-21  dashboard@demo.com        121871  33:51:11
1 2019-01-22  dashboard@demo.com         69832  19:23:52
2 2019-01-23  dashboard@demo.com         35247   9:47:27
3 2019-01-24  dashboard@demo.com         38405  10:40:05
4 2019-01-25  dashboard@demo.com         83725  23:15:25

In [63]:
grouped = grouped[::-1]
grouped['Day'] = grouped['Date'].dt.dayofweek
grouped['DayName'] = grouped['Day'].map({0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
grouped

Date                   email  TotalSeconds TimeSpent  Day    DayName
512 2020-08-18  sethi.sankalp@karam.in         43549  12:05:49    1    Tuesday
511 2020-08-17         mayank@karam.in         18171   5:02:51    0     Monday
510 2020-08-13      dhanirams@karam.in         33356   9:15:56    3   Thursday
509 2020-08-12      rohitrana@karam.in         14274   3:57:54    2  Wednesday
508 2020-08-08  ashish.pandey@karam.in         55056  15:17:36    5   Saturday
..         ...                     ...           ...       ...  ...        ...
4   2019-01-25      dashboard@demo.com         83725  23:15:25    4     Friday
3   2019-01-24      dashboard@demo.com         38405  10:40:05    3   Thursday
2   2019-01-23      dashboard@demo.com         35247   9:47:27    2  Wednesday
1   2019-01-22      dashboard@demo.com         69832  19:23:52    1    Tuesday
0   2019-01-21      dashboard@demo.com        121871  33:51:11    0     Monday

[513 rows x 6 columns]

In [64]:
filter = grouped["Day"]==0
onlymonday = grouped.head().where(filter)
onlymonday.dropna(inplace=True)
onlymonday

Date            email  TotalSeconds TimeSpent  Day DayName
511 2020-08-17  mayank@karam.in       18171.0   5:02:51  0.0  Monday

In [65]:
import datetime
recentmonday = onlymonday['Date']
print(recentmonday)
lastmonday = pd.to_datetime(recentmonday) - datetime.timedelta(days=7)
print(lastmonday)

511   2020-08-17
Name: Date, dtype: datetime64[ns]
511   2020-08-10
Name: Date, dtype: datetime64[ns]


In [66]:
recentmonday = str(recentmonday).split()[1]
lastmonday = str(lastmonday).split()[1]
mask = (grouped['Date'] > lastmonday) & (grouped['Date'] <= recentmonday)
grouped = grouped[mask]
grouped = grouped[['Date','DayName','email','TimeSpent']]

In [67]:
grouped[::-1]

Date    DayName               email TimeSpent
509 2020-08-12  Wednesday  rohitrana@karam.in   3:57:54
510 2020-08-13   Thursday  dhanirams@karam.in   9:15:56
511 2020-08-17     Monday     mayank@karam.in   5:02:51